## Harvard Dataset EDA
The Harvard dataset, which contains information on individual candidates, aims to enrich the exiting individual dataset (`individuals_table.csv`) from various states. (However, as Sarah suggested, it's still uncertain whether a new table for candidates should be created. If so, it could include more variables.) For now, I've selected variables that could 1) help match individuals within the two datasets and 2) provide additional information in addition to the individual's table, such as election outcomes and candidate party affiliations.

This Markdown document is designed to explain the rationale behind the selection of variables within the Harvard dataset and to describe the basic nature of these variables.

In [1]:
import pandas as pd
import numpy as np

# TODO: #98 notebooks should be in notebooks/ folder

In [2]:
hv_df = pd.read_csv("harvard.csv")
# TODO: #97 this is not runnable as there are no instructions on how to get 'harvard.csv'

FileNotFoundError: [Errno 2] No such file or directory: 'harvard.csv'

In [ ]:
HV_INDIVIDUAL_COLS = [
    "caseid",
    "year",
    "month",
    "day",
    "sab",
    "cname",
    "candid",
    "cand",
    "sen",
    "partyz",
    "partyt",
    "outcome",
    "last",
    "first",
    "v19_20171211",
    "v19_20160217"

]
raw_df = hv_df[HV_INDIVIDUAL_COLS]

1. Name columns (`v19_20171211`,`v19_20160217`,`cand`,
`last` and `first`)  
`cand`	Standardized Candidate Name  
`v19_20171211`	Standardized Candidate Name from December 11, 2017  
`v19_20160217`	Standardized Candidate Name from February 17, 2016  


In [ ]:
# if two standardized candidate columns match
# What is the pattern for update, any correction of just update from NaN.
raw_df[
    (raw_df["v19_20171211"].str.strip() != raw_df["v19_20160217"].str.strip()) &
    ~raw_df["v19_20160217"].isna()
][["v19_20171211", "v19_20160217"]]


,v19_20171211,v19_20160217


We thus know that v19_20171211 is a update of v19_20160217, there is no additional information within the v19_20160217 volumn. 

In [ ]:
raw_df["v19_20171211"] = raw_df["v19_20171211"].str.lower()
raw_df[
    (raw_df["cand"].str.strip() != raw_df["v19_20171211"].str.strip()) & 
    raw_df["v19_20171211"].isna()
][["v19_20171211", "cand","first","last"]]

/tmp/ipykernel_11723/3528258281.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df["v19_20171211"] = raw_df["v19_20171211"].str.lower()


,v19_20171211,cand,first,last
50551,NaN,"fullwood, reggie",reggie,fullwood
50552,NaN,"jefferson, lawrence m. (jeff)",lawrence,jefferson
50553,NaN,"grant, james",james,grant
50554,NaN,"matthews, daniel john",daniel,matthews
73338,NaN,"mcalister, kate",kate,mcalister
...,...,...,...,...
168820,NaN,namemissing7,NaN,NaN
168862,NaN,namemissing8,NaN,NaN
168867,NaN,namemissing9,NaN,NaN
168868,NaN,namemissing10,NaN,NaN


So, there are two things to do 1) update the `cand` column that use the non-empty v19 values to replace the cand (full_name) column; 2) delete all rows with missingnames. 

In [ ]:
raw_df[raw_df["cand"].isna()]

,caseid,year,month,day,sab,cname,candid,cand,sen,partyz,partyt,outcome,last,first,v19_20171211,v19_20160217


In [ ]:
raw_df['cand'] = np.where(raw_df['v19_20171211'].notna(),raw_df["v19_20171211"],raw_df["cand"])
raw_df = raw_df[~raw_df['cand'].str.startswith('namemissing')]


In [ ]:
raw_df[(raw_df["last"].isna() & raw_df["first"].isna())]

,caseid,year,month,day,sab,cname,candid,cand,sen,partyz,partyt,outcome,last,first,v19_20171211,v19_20160217


2. `sab` column (state)  
We only need to uppercase every value here.

In [ ]:
raw_df["sab"] = raw_df["sab"].str.upper()
print(raw_df["sab"].unique())

['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'DE' 'FL' 'GA' 'HI' 'ID' 'IL' 'IN'
 'IA' 'KS' 'KY' 'LA' 'ME' 'MD' 'MA' 'MI' 'MN' 'MS' 'MO' 'MT' 'NE' 'NV'
 'NH' 'NJ' 'NM' 'NY' 'NC' 'ND' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD' 'TN'
 'TX' 'UT' 'VT' 'VA' 'WA' 'WV' 'WI' 'WY']


3. Candidate uniqueness?
In order to merge the datasets, we need to ensure the uniqueness of each individual. That is, name/id of each observation is unique. 

In [ ]:
# not unique? 
raw_df["candid"].value_counts()

candid
167521    100
168980     77
167044     73
165078     72
166643     66
         ... 
5236        1
5264        1
357547      1
357550      1
357552      1
Name: count, Length: 160377, dtype: int64

In [ ]:
unique_party_counts = raw_df.groupby("candid")["partyt"].nunique().reset_index(name="unique_partyt_count")

unique_party_counts[unique_party_counts["unique_partyt_count"] > 1]

,candid,unique_partyt_count
7,8,2
20,21,2
29,30,2
52,53,2
58,60,2
...,...,...
153452,354159,2
153609,354938,2
155438,363848,2
155445,363876,2


In [ ]:
raw_df[raw_df["candid"] == 8]

,caseid,year,month,day,sab,cname,candid,cand,sen,partyz,partyt,outcome,last,first,v19_20171211,v19_20160217
6730,779,1986,11.0,4.0,AK,NaN,8,"donley, dave",0,d,d,w,donley,dave,"donley, dave","DONLEY, DAVE"
6841,783,1988,11.0,8.0,AK,NaN,8,"donley, dave",0,d,d,w,donley,dave,"donley, dave","DONLEY, DAVE"
6946,732,1990,11.0,6.0,AK,NaN,8,"donley, dave",0,d,d,w,donley,dave,"donley, dave","DONLEY, DAVE"
8544,90,1992,11.0,3.0,AK,NaN,8,"donley, dave",1,d,d,w,donley,dave,"donley, dave","DONLEY, DAVE"
8580,160,1994,11.0,8.0,AK,NaN,8,"donley, dave",1,d,d,w,donley,dave,"donley, dave","DONLEY, DAVE"
8620,161,1998,11.0,3.0,AK,NaN,8,"donley, dave",1,r,r,w,donley,dave,"donley, dave","DONLEY, DAVE"
8685,8,2002,11.0,5.0,AK,NaN,8,"donley, dave",1,r,r,l,donley,dave,"donley, dave","DONLEY, DAVE"


For one candidate, they may have several election records. They may have several election results at different counties. Throughout their career, they may have different parties.

4. Other variables

`partyt` Assigns just one party to a candidate in one election season (i.e., the primary and general election in one year), using the same seven codes used in partyz.  
	For example, a candidate running in NY with fusion as a Democrat and a Republican is assigned the party they are expected to caucus with in the state legislature, measured by how the end up caucusing in the state legislature.  
	For example, a candidate who files in a state primary as a Democrat, and then is written in by voters in the Republican primary, has “d” designated as their “true” party.  


In [ ]:
raw_df["partyt"].value_counts()

partyt
d            183874
r            151981
nonmaj        22189
writein       17574
nonpart        2431
partymiss       268
Name: count, dtype: int64

In [ ]:
mini_df = pd.read_csv("/project/data/transformed/inds_mini.csv")

In [ ]:
mini_df["party"].value_counts()

party
republican    2
democratic    1
DELETE        1
Name: count, dtype: int64

We rename the party categories to align with the individual file.

2015 -2017 data
create a new table for elections who was runnning, foreign key to ind's table 
what race they are 
district, vote, results + # of votes -> how many more votes etc.


linkage -> file, to find match individual, for the election results -> look for potential match -> include in the election results. 

statefinancetransformer -> new class -> election result transformer 

1. cleaning the data 
2. doing matching -> reuse the funciton in linkage 
